In [1]:
from segmentizer import Segmentizer
from segmentizer.data_loader import LASIESTADataLoader
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from tqdm import tqdm_notebook, tnrange
from IPython.display import clear_output

In [2]:
data_loader = LASIESTADataLoader('/Users/dsoellinger/Downloads/I_SI_01')
video_segmentizer = Segmentizer(352,288)

## Warm up phase (Train the model)

In [3]:
nr_frames_to_train = 100
frame_rate = 5    # Every frames is used x times during training
train_idx = 0

print('Start training - Using the first ' + str(nr_frames_to_train) + ' frames to train')
      
frame = data_loader.get_next_frame()
train_idx += 1
      
for _ in tnrange(nr_frames_to_train):
    for _ in range(frame_rate):
        video_segmentizer.fit(frame, init_weight=0.03, init_variance=36.0, lr=0.005)
      
    frame = data_loader.get_next_frame()
    train_idx += 1

Start training - Using the first 100 frames to train


HBox(children=(IntProgress(value=0), HTML(value='')))

KeyboardInterrupt: 

In [4]:
data_loader.set_next_frame_idx(0)

background_fig = plt.figure()
background_fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
plt.axis('off')

def animate(*args):
    clear_output(wait=True)
    print("Process frame: " + str(data_loader.get_next_frame_idx()))
    frame = data_loader.get_next_frame()
    background = video_segmentizer.fit_and_predict(frame, init_weight=0.03, init_variance=36.0, lr=0.005)
    return plt.imshow(background, animated=True, aspect='auto', vmin=0, vmax=1, cmap='gray')


anim = animation.FuncAnimation(background_fig, animate, frames=data_loader.get_nr_of_frames()-1)
anim.save('background_estimate_I_SI_01.mp4', fps=20, extra_args=['-vcodec', 'libx264'])
print("Done")

Process frame: 0


TypeError: Invalid dimensions for image data

<video controls src="background_estimate_I_SI_01.mp4" />